以下面为例：

$$
f(x1,x2)=ln(x1)+x1x2−sin(x2) \tag{1}
$$

因为是基于 ADTangent 类进行了操作符重载，因此在初始化自变量 x 和 y 的值需要使用 ADTangent 来初始化，然后通过代码 f = ADTangent.log(x) + x * y - ADTangent.sin(y) 来实现。

由于这里是求 f 关于自变量 x 的导数，因此初始化数据的时候呢，自变量 x 的 dx 设置为1，而自变量 y 的 dx 设置为0。

In [1]:
import numpy as np

In [2]:
class ADTangent:
    def __init__(self, x, dx):
        self.x = x
        self.dx = dx

    def __str__(self):
        context = f'value: {self.x:.4f}, grad:{self.dx:.4f}'
        return context
    
    def __add__(self, other):
        if isinstance(other, ADTangent):
            x = self.x + other.x
            dx = self.dx + other.dx
        elif isinstance(other, float):
            x = self.x + other
            dx = self.dx
        else:
            return NotImplementedError
        return ADTangent(x, dx)
    
    def __sub__(self, other):
        if isinstance(other, ADTangent):
            x = self.x - other.x
            dx = self.dx - other.dx
        elif isinstance(other, float):
            x = self.x - other
            dx = self.dx
        else:
            return NotImplementedError
        return ADTangent(x, dx)
    
    def __mul__(self, other):
        if isinstance(other, ADTangent):
            x = self.x * other.x
            dx = self.x * other.dx + self.dx * other.x
        elif isinstance(other, float):
            x = self.x * other
            dx = self.dx * other
        else:
            return NotImplementedError
        return ADTangent(x, dx)
    
    def log(self):
        x = np.log(self.x)
        dx = self.dx / self.x
        return ADTangent(x, dx)
    
    def sin(self):
        x = np.sin(self.x)
        dx = self.dx * np.cos(self.x)
        return ADTangent(x, dx)            

In [3]:
x1 = ADTangent(x=2., dx=1)
x2 = ADTangent(x=5., dx=0)
f = ADTangent.log(x1) + x1 * x2 - ADTangent.sin(x2)
print(f)

value: 11.6521, grad:5.5000


In [4]:
import torch
from torch.autograd import Variable

x = Variable(torch.tensor(2.), requires_grad=True)
y = Variable(torch.tensor(5.), requires_grad=True)
f = torch.log(x) + x * y - torch.sin(y)
f.backward()

print(f)
print(x.grad)

tensor(11.6521, grad_fn=<SubBackward0>)
tensor(5.5000)


In [5]:
import numpy as np
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Tensor, Parameter

class Fun(nn.Cell):
    def __init__(self):
        super(Fun, self).__init__()

    def construct(self, x, y):
        f = ops.log(x) + x * y - ops.sin(y)
        return f
    
x = Tensor(np.array([2.], np.float32))
y = Tensor(np.array([5.], np.float32))
f = Fun()(x, y)
grad_all = ops.GradOperation()
grad = grad_all(Fun())(x, y)

print(f)
print(grad[0])

[WARNING] ME(648955:140529887967040,MainProcess):2024-03-24-20:40:28.882.210 [mindspore/run_check/_check_version.py:102] MindSpore version 2.2.11 and cuda version 11.4.148 does not match, CUDA version [['10.1', '11.1', '11.6']] are supported by MindSpore officially. Please refer to the installation guide for version matching information: https://www.mindspore.cn/install.


[11.652072]
5.5
